# Here we look at (extended) dynamic mode composition, and its relation to Koopman

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#### The first toy example is taken from Clarence Rowley's talk https://ipam.wistia.com/medias/sl5hs1srxf (~21:30)
We start with the following $z=F(z)$ dynamics
$$
\begin{bmatrix} z_1 \\
                z_2 
                \end{bmatrix}
=
\begin{bmatrix} \lambda z_1 \\
               \mu z_2 + (\lambda^2 - \mu)cz_1^2
               \end{bmatrix}
$$
The Koopman operator $U$ is then defined on observables by $U\phi = \phi \circ F$. \
The eigenvalues of $U$ are $\lambda$ and $\mu$ and their respective eigenfunctions (eigenobservables) are 
$$ \phi_\lambda(z) = z_1 $$
$$ \phi_\mu(z) = z_2 - cz_1^2 $$
We can use DMD to recover the eigenvalues.

We start with a dataset of points $Z = \begin{bmatrix} z^{(1)} z^{(2)} \dots z^{(n-1)} \end{bmatrix}$, and let $Z' = F(Z) = \begin{bmatrix} z^{(2)} z^{(3)} \dots z^{(n)} \end{bmatrix}$ \
We form the observation matrices $X = g(Z)$, $Y = g(Z')$ \
And then learn $A$ to relate these by $Y=AX$. \
It then turns out that the eigenvalues of $A$ are also eigenvalues of $U$!

In [174]:
# Dynamics (forward map) function
def F(z, lmbda=0.9, mu=0.5, c=1):
    z1,z2 = z
    newz = [lmbda*z1, mu*z2 + (lmbda**2 - mu)*c*z1**2]
    return np.array(newz)
def obs(z,obs_dct):
    observations = [g(z) for g in obs_dct]
    return np.array(observations)
obs_dct1 = {
    lambda z: z[0],
    lambda z: z[1]
}
obs_dct2 = {
    lambda z: z[0],
    lambda z: z[1],
    lambda z: z[0]**2
}
obs_dct3 = {
    lambda z: z[0],
    lambda z: z[1],
    lambda z: z[1]**2
}
Z = np.array([[-5,5],[-1,1],[1,1],[5,5]]).T # data points
# But different datasets give different results! (sometimes better, sometimes worst)
# Z = np.array([[-5,-5],[-1,-1],[-1,1],[-5,5]]).T
# Z = np.array([[-5,-5],[-1,-1],[-1,1],[-5,5]]).%

In [175]:
# First we test the linear case (c=0), which looks like
# F = [0.9   0]
#     [0.  0.5]
obs_dct = obs_dct1 # choose dictionary of observables
FZ = np.apply_along_axis(lambda z: F(z,c=0),0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print("Eigenvalues discoveredD) # sure enough we recover the eigenvalues (but other datasets may fail!)

[0.9 0.5]


In [177]:
# Then we test with c=1 (doesn't work because observables (z1,z2) not rich enough!)
obs_dct = obs_dct1 # choose dictionary of observables
FZ = np.apply_along_axis(F,0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print(D)

[0.9        2.00230769]


In [179]:
# Now we try with 2nd dictionary of observables (z1,z2,z1^2)
obs_dct = obs_dct2 # choose dictionary of observables
FZ = np.apply_along_axis(F,0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print(D)

[0.9  0.5  0.81]


In [181]:
# Now we try with 3rd dictionary of observables (z1,z2,z2^2) -- z2^2 is  not good enough to recover 2nd eval
obs_dct = obs_dct3 # choose dictionary of observables
FZ = np.apply_along_axis(F,0,Z)
X = np.apply_along_axis(lambda z: obs(z,obs_dct),0,Z)
Y = np.apply_along_axis(lambda z: obs(z,obs_dct),0,FZ)

# We learn A st Y=AX
A = Y @ np.linalg.pinv(X)
D,Q = np.linalg.eig(A)
print(D)

[0.9        0.82205673 4.76704327]
